In [88]:
# Helper libraries
sys.path.append('a3_framework')
import a3utils, a3tf_embed_viz, a3rnnlm
reload(a3utils)
reload(a3tf_embed_viz)
reload(a3rnnlm)

import util, rnnlm
reload(util)
reload(rnnlm)

import json, os, re, shutil, sys, time
import collections, itertools
import sys

from IPython.display import display, HTML

# NLTK for NLP utils and corpora
import nltk

# NumPy and TensorFlow
import numpy as np
import tensorflow as tf
assert(tf.__version__.startswith("0.12"))

# utils.pretty_print_matrix uses Pandas. Configure float format here.
import pandas as pd
pd.set_option('float_format', lambda f: "{0:.04f}".format(f))

print "Done!"



Done!


In [89]:
samp = True
samp = '_samp' if samp else ''

In [90]:
# Load feature names
fnames = np.array(pd.read_pickle('../interim/028_preproc_heavy_shows_concat' + samp + '.p'))

# load concatenated descriptions
desc = np.array(pd.read_pickle('../interim/028_preproc_heavy_show_description_concat' + samp + '.p'))

print("Feature Names Shape:", fnames.shape)
print("Descriptions Shape:", desc.shape)
assert fnames.shape[0] == desc.shape[0]

('Feature Names Shape:', (1192, 2))
('Descriptions Shape:', (1192, 1))


In [91]:
reload(rnnlm)

TF_GRAPHDIR = "tf_graph"

# Clear old log directory.
shutil.rmtree(TF_GRAPHDIR, ignore_errors=True)

lm = rnnlm.RNNLM(V=10000, H=200, num_layers=2)
lm.BuildCoreGraph()
lm.BuildTrainGraph()
lm.BuildSamplerGraph()

summary_writer = tf.summary.FileWriter(TF_GRAPHDIR, lm.graph)
print "Done!"

Done!


In [98]:
# Load the dataset
reload(a3utils)
reload(a3rnnlm)

#training parameters
training_params = dict(max_time=20, batch_size=50, learning_rate=0.5, num_epochs=2)

# Model parameters
vocab_size = 10000
model_params = dict(V=vocab_size, H=100, softmax_ns=200, num_layers=1)

output = []
for i in range(0, desc.shape[0]):
    vocab, train_ids, test_ids = a3utils.load_corpus(desc[i][0], split=0.8, V=vocab_size, shuffle=42)
    final_h = a3rnnlm.execute_rnnlm(model_params, training_params, vocab, train_ids, test_ids)
    output.append(final_h.ravel())
    if i >= 2: break;

print np.asarray(output).shape

(91,)
Loaded 91 sentences (1514 tokens)
Training set: 72 sentences (1163 tokens)
Test set: 19 sentences (351 tokens)
[epoch 1] Starting epoch 1
[epoch 1] Completed in 0:00:00
[epoch 1] Train set: avg. loss: 8.777  (perplexity: 6486.36)
[epoch 1] Test set: avg. loss: 8.804  (perplexity: 6659.12)

[epoch 2] Starting epoch 2
[epoch 2] Completed in 0:00:00
[epoch 2] Train set: avg. loss: 8.319  (perplexity: 4101.44)
[epoch 2] Test set: avg. loss: 8.448  (perplexity: 4667.27)

(688,)
Loaded 688 sentences (11082 tokens)
Training set: 550 sentences (8992 tokens)
Test set: 138 sentences (2090 tokens)
[epoch 1] Starting epoch 1
[epoch 1] Completed in 0:00:01
[epoch 1] Train set: avg. loss: 7.532  (perplexity: 1867.70)
[epoch 1] Test set: avg. loss: 7.535  (perplexity: 1871.77)

[epoch 2] Starting epoch 2
[epoch 2] Completed in 0:00:01
[epoch 2] Train set: avg. loss: 7.492  (perplexity: 1793.56)
[epoch 2] Test set: avg. loss: 7.423  (perplexity: 1674.28)

(70,)
Loaded 70 sentences (2910 tokens)


[epoch 1] Starting epoch 1
[epoch 1] Completed in 0:00:00
[epoch 1] Train set: avg. loss: 8.663  (perplexity: 5787.25)
[epoch 1] Test set: avg. loss: 8.786  (perplexity: 6538.92)

[epoch 2] Starting epoch 2
[epoch 2] Completed in 0:00:00
[epoch 2] Train set: avg. loss: 8.293  (perplexity: 3994.10)
[epoch 2] Test set: avg. loss: 8.475  (perplexity: 4793.44)

(100, 100)
